In [1]:
import pandas as pd
import numpy as np
import math
import datetime

table=pd.read_csv("../Dati/orari/route.csv", delimiter=";")
time_table=pd.read_csv("../Dati/orari/route_time.csv", delimiter=";")
dft = pd.read_csv("../Dati/distanze/times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
fur1 = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";")
fur2 = pd.read_csv("../Dati/turni_f/furgoncino2_routing.csv", delimiter=";")

def get_row(df, string):
    return pd.Series(df.index).str.contains(string, case=False)

def get_column(df, string):
    return pd.Series(df.columns).str.contains(string, case=False)

def difference(day, index, index2):
    time = datetime.datetime.strptime(time_table[day][index], '%H:%M')
    time2 = datetime.datetime.strptime(time_table[day][index2], '%H:%M')
    return ((time2-time).seconds)/60


In [49]:
def new_route(schedule, day):
    #day='lunedi'

    times = []
    keys = []
    row = dft[get_row(dft, 'Cremona, sesto 39').values]
    key1 = ''
    key2 = ''
    impossibile = []
    for i in range(1,len(schedule)-1):
        key1 = row.index[0]
        try:
            paese=schedule[i]
        except:
            continue
        #Distanze in tempo dal paese corrente di ognuno dei furgoni alla destinazione
        value=row.loc[:,get_column(dft, paese).values].values[0][0]
        times.append(round(value))
        row = dft[get_row(dft, paese).values]
        key2 = row.index[0]
        keys.append(key1 + ' - ' + key2)
    
    steps_times1 = pd.Series(times , keys)
    #creo una copia della serie
    #steps_times2 = steps_times1.copy()
    #creo una copia dei tempi
    times1 = times[:]
    #times2 = times[:]

    #aggiungo la chiave per l'ultima destinazione considerata e la casa dell'autista

    #autista 1
    row = dft[get_row(dft, paese).values]
    value=row.loc[:,get_column(dft, 'Ostiano, Mazzini, 1').values].values[0][0]
    s1 = pd.Series(round(value), [row.index[0] + ' - Ostiano, Mazzini, 1'])
    steps_times1 = steps_times1.append(s1)
    times1.append(round(value))

    """#autista 2
    value=row.loc[:,get_column(dft, 'San Giovanni in Croce, Feudatari, 25 A').values].values[0][0]
    s2 = pd.Series(round(value), [row.index[0] + ' - San Giovanni in Croce, Feudatari, 25 A'])
    steps_times2 = steps_times2.append(s2)
    times2.append(round(value))

    if(sum(times1) < sum(times2)):"""
    return steps_times1
    """else:
        return steps_times2"""

In [3]:
#first attempt to local search algorithm
def find_best_position(steps_times, to_insert):
    times = []
    key2 = []
    for i in range(len(steps_times)):
        cities = steps_times.keys()[i].split('-')
        row1 = dft[get_row(dft, cities[0].strip()).values]
        #time from city 1 to current (to insert)
        f_c1_to_curr = row1.loc[:,get_column(dft, to_insert).values].values[0][0]
        row2 = dft[get_row(dft, to_insert).values]
        #time from current (to insert) to city 2
        f_curr_to_c2 = row2.loc[:,get_column(dft, cities[1].strip()).values].values[0][0]
        times.append(f_c1_to_curr + f_curr_to_c2)
    s3 = pd.Series(times, steps_times.keys())
    return s3

In [46]:
#furgoncino 1 (MON, FRI)
def local_search1(i, day):
    fur1 = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";")

    schedule = []
    schedule.append('Cremona, Sesto 39'.lower())
    for j in range(len(fur1[day].dropna())):
        schedule.append(dft[get_row(dft, fur1[day][j]).values].index[0].lower())

    schedule.append('Ostiano, Mazzini, 1'.lower())
    #schedule.append('San Giovanni in Croce, Feudatari, 25 A'.lower())

    #print(schedule)
    l = len(schedule)-2
    moved = []
    print(schedule[i])
    #finché in moved non ci sono tutte le città (meno Cremona e Ostiano)
    while len(moved) < l:
        #faccio il pop della città
        to_insert = schedule.pop(i)
        #controllo se l'ho già considerata
        if(to_insert not in moved):
            #(se no) la inserisco nella lista di quelle già considerate
            moved.append(to_insert)
            #calcolo la nuova route tra le città rimanenti
            route = new_route(schedule, day)
            #trovo la posizione migliore
            best = find_best_position(route, to_insert).idxmin()
            city = best.split('-')[0].strip().lower()
            try:
                #trovo l'indice della posizione migliore
                position = schedule.index(city) + 1
            except:
                #problema nome diverso
                if(city == 'casale cremasco'):
                    city = 'casale cremasco-vidolasco, roma, 56'
                    position = schedule.index(city) + 1
            #inserisco la città nella posizione corretta
            schedule.insert(position ,to_insert)
        else:
            #(se si) reinserisco la città nella sua posizione e vado avanti
            schedule.insert(i ,to_insert)
            #incremento l'indice per la prossima città
            i += 1
            i = i%(l+1)
            if(i == 0):
                i = 1
    return schedule

In [47]:
#returns a pd.Series: keys = city1 - city2, values = times from city1 to city2
def schedule_time(new_schedule):
    times = []
    row = dft[get_row(dft, 'Cremona, sesto 39').values]
    keys = []
    key1 = ''
    key2 = ''
    for i in range(1,len(new_schedule)):
        key1 = row.index[0]
        paese=new_schedule[i]

        value=row.loc[:,get_column(dft, paese).values].values[0][0]
        times.append(round(value))
        
        row = dft[get_row(dft, paese).values]
        key2 = row.index[0]
        keys.append(key1 + ' - ' + key2)
    steps_times = pd.Series(times , keys)
    return steps_times

In [52]:
fur1 = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";")
day = 'venerdi'
schedule = []
schedule.append('Cremona, Sesto 39'.lower())
for i in range(len(fur1[day].dropna())):
    schedule.append(dft[get_row(dft, fur1[day][i]).values].index[0].lower())
schedule.append('Ostiano, Mazzini, 1'.lower())
    
best_serie = schedule_time(schedule)
best_time = sum(best_serie.values)

for i in range(1, len(schedule)-1):
    new_schedule = local_search1(i, day)
    time = sum(schedule_time(new_schedule).values)
    print(time)
    if(time < best_time):
        best_time = time
        best_serie = new_schedule
        print('scambiato')
        
print(best_time)
print(best_serie)

crema, civerchi, 9
191.0
capergnanica, xi febbraio, 7
192.0
bagnolo cremasco, roma, 3
200.0
vaiano cremasco,  gloriosi caduti, 3
196.0
monte cremasco, roma, 12
192.0
dovera, oldrini 2
192.0
pandino, castello, 15
192.0
agnadello, marconi, 6
191.0
rivolta d'adda, stefano renzi, 5
193.0
spino d'adda, circonvallazione, 13
193.0
trescore cremasco, vittorio veneto, 1
205.0
casaletto vaprio, maggiore, 7
193.0
180.0
Cremona, Sesto 39 - Crema, Civerchi, 9                                        33.0
Crema, Civerchi, 9 - Capergnanica, XI febbraio, 7                             11.0
Capergnanica, XI febbraio, 7 - Bagnolo Cremasco, Roma, 3                       9.0
Bagnolo Cremasco, Roma, 3 - Vaiano Cremasco,  Gloriosi Caduti, 3               6.0
Vaiano Cremasco,  Gloriosi Caduti, 3 - Monte Cremasco, Roma, 12                3.0
Monte Cremasco, Roma, 12 - Dovera, Oldrini 2                                   8.0
Dovera, Oldrini 2 - Pandino, Castello, 15                                      8.0
Pandino

In [40]:
s1 = local_search1(1)
print(s1)
print(sum(schedule_time(s1).values))

sesto ed uniti, roma, 4
['cremona, sesto 39', 'sesto ed uniti, roma, 4', 'pizzighettone, giuseppe garibaldi, 18', 'grumello cremonese ed uniti, roma, 2', 'soresina, giacomo matteotti, 6', 'soncino, dante alighieri, 23', 'trigolo, roma, 26', 'castelleone, roma, 67', 'ripalta cremasca, roma, 5', 'montodine,  xxv aprile, 10', 'ricengo, guglielmo marconi', 'capralba, piave, 2', 'sergnano, iv novembre,  10', 'casale cremasco-vidolasco, roma, 56', 'camisano, trieste, 57', 'ostiano, mazzini, 1']
209.0


In [41]:
s2 = local_search1(2)
print(s2)
print(sum(schedule_time(s2).values))

grumello cremonese ed uniti, roma, 2
['cremona, sesto 39', 'pizzighettone, giuseppe garibaldi, 18', 'sesto ed uniti, roma, 4', 'grumello cremonese ed uniti, roma, 2', 'soresina, giacomo matteotti, 6', 'soncino, dante alighieri, 23', 'trigolo, roma, 26', 'castelleone, roma, 67', 'ripalta cremasca, roma, 5', 'montodine,  xxv aprile, 10', 'ricengo, guglielmo marconi', 'capralba, piave, 2', 'sergnano, iv novembre,  10', 'casale cremasco-vidolasco, roma, 56', 'camisano, trieste, 57', 'ostiano, mazzini, 1']
210.0


In [71]:
#furgoncino 1 (MON, FRI)
def local_search2(i, day):
    fur1 = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";")

    schedule = []
    schedule.append('Cremona, Sesto 39'.lower())
    for j in range(len(fur1[day].dropna())):
        schedule.append(dft[get_row(dft, fur1[day][j]).values].index[0].lower())

    schedule.append('Ostiano, Mazzini, 1'.lower())
    #schedule.append('San Giovanni in Croce, Feudatari, 25 A'.lower())

    schedule_pre = schedule[:]
    schedule_pre_time = sum(schedule_time(schedule_pre).values)
    #print(f"first pre time: {schedule_pre_time}")
    #print(schedule)
    l = len(schedule)-2
    moved = []
    print(schedule[i])
    #finché in moved non ci sono tutte le città (meno Cremona e Ostiano)
    while len(moved) < l:
        #faccio il pop della città
        to_insert = schedule.pop(i)
        #controllo se l'ho già considerata
        if(to_insert not in moved):
            #(se no) la inserisco nella lista di quelle già considerate
            moved.append(to_insert)
            #calcolo la nuova route tra le città rimanenti
            route = new_route(schedule, day)
            #trovo la posizione migliore
            best = find_best_position(route, to_insert).idxmin()
            city = best.split('-')[0].strip().lower()
            try:
                #trovo l'indice della posizione migliore
                position = schedule.index(city) + 1
            except:
                #problema nome diverso
                if(city == 'casale cremasco'):
                    city = 'casale cremasco-vidolasco, roma, 56'
                    position = schedule.index(city) + 1
            #inserisco la città nella posizione corretta
            schedule.insert(position ,to_insert)
            new_time = sum(schedule_time(schedule).values)
            #print(new_time)
            if(new_time < schedule_pre_time):
                schedule_pre = schedule[:]
                schedule_pre_time = new_time
                #print('scambiato')
            else:
                schedule = schedule_pre[:]
        else:
            #(se si) reinserisco la città nella sua posizione e vado avanti
            schedule.insert(i ,to_insert)
            #incremento l'indice per la prossima città
            i += 1
            i = i%(l+1)
            if(i == 0):
                i = 1
    return schedule_pre

In [74]:
fur1 = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";")
day = 'venerdi'
schedule = []
schedule.append('Cremona, Sesto 39'.lower())
for i in range(len(fur1[day].dropna())):
    schedule.append(dft[get_row(dft, fur1[day][i]).values].index[0].lower())
schedule.append('Ostiano, Mazzini, 1'.lower())
    
best_serie = schedule_time(schedule)
best_time = sum(best_serie.values)

for i in range(1, len(schedule)-1):
    new_schedule = local_search2(i, day)
    time = sum(schedule_time(new_schedule).values)
    print(time)
    if(time < best_time):
        best_time = time
        best_serie = new_schedule[:]
        print('scambiato')
        
print(best_time)
print(best_serie)

crema, civerchi, 9
180.0
capergnanica, xi febbraio, 7
180.0
bagnolo cremasco, roma, 3
180.0
vaiano cremasco,  gloriosi caduti, 3
180.0
monte cremasco, roma, 12
180.0
dovera, oldrini 2
180.0
pandino, castello, 15
180.0
agnadello, marconi, 6
180.0
rivolta d'adda, stefano renzi, 5
180.0
spino d'adda, circonvallazione, 13
180.0
trescore cremasco, vittorio veneto, 1
180.0
casaletto vaprio, maggiore, 7
180.0
180.0
Cremona, Sesto 39 - Crema, Civerchi, 9                                        33.0
Crema, Civerchi, 9 - Capergnanica, XI febbraio, 7                             11.0
Capergnanica, XI febbraio, 7 - Bagnolo Cremasco, Roma, 3                       9.0
Bagnolo Cremasco, Roma, 3 - Vaiano Cremasco,  Gloriosi Caduti, 3               6.0
Vaiano Cremasco,  Gloriosi Caduti, 3 - Monte Cremasco, Roma, 12                3.0
Monte Cremasco, Roma, 12 - Dovera, Oldrini 2                                   8.0
Dovera, Oldrini 2 - Pandino, Castello, 15                                      8.0
Pandino